In [ ]:
import sc3nb as scn
from PIL import Image, ImageDraw, ImageFont

In [ ]:
image = Image.new("RGB", (240, 240), (0, 0, 0))
draw = ImageDraw.Draw(image)
font = ImageFont.truetype("AtkinsonHyperlegible-Regular.ttf", 19)
small_font = ImageFont.truetype("AtkinsonHyperlegible-Regular.ttf", 12)
colour_palette = ["#65A1F6","#4DA167","#52489C","#EF6F6C"]
sine_img = Image.open("Sine_wave.png")
white_img = Image.open("White_Noise.png")
pink_img = Image.open("Pink_Noise.png")


sc = scn.startup()


In [ ]:
waves = ["Sine","Triangle","Square","Sawtooth","Inv Saw","White","Pink","Octaves","1/2 Oct","1/3 Oct","Semitone","Whole Tone"]
wave_freq = 1000
oct_freq = 440
level = -6
wave_index = 0
resolution = 0 # [Coarse, Fine, Finest]

active_param = 0


In [ ]:
def cycle_wave(inc = True):
    global wave_index
    wave_index += 1 if inc else -1
    wave_index %= len(waves)
    play()

def cycle_freq(inc = True):
    global wave_freq, oct_freq, active_freq
    if wave_index < 5:
        match resolution:
            case 0:
                wave_freq += 100 if inc else -100
            case 1:
                wave_freq += 10 if inc else -10
            case 2:
                wave_freq += 1 if inc else -1
        wave_freq %=20000
        
        
    elif wave_index > 6:
        match resolution:
            case 0:
                oct_freq += 100 if inc else -100
            case 1:
                oct_freq += 10 if inc else -10
            case 2:
                oct_freq += 1 if inc else -1
        oct_freq %=20000
        
  
    play()

def cycle_level(inc = True):
    global level
    match resolution:
        case 0:
            level += 6 if inc else -6
        case 1:
            level += 1 if inc else -1
        case 2:
            level += 0.1 if inc else -0.1
    level = min(0,level)
    play()

In [ ]:
def draw_octave_lines(num_lines):
    draw.line([15,115,185,115],width=2)
    for i in range(num_lines):
        draw.line([((i+1)*170/(num_lines+1)+15),115,(((i+1)*170/(num_lines+1))+15),25],width=2)



def render_display():

    draw.rectangle([0,0,200,160],fill=colour_palette[0])
    draw.rectangle([0,160,200,200],fill=colour_palette[1])
    draw.rectangle([0,200,200,240],fill=colour_palette[2])
    # Parameter seperators
    draw.line([0,160,200,160])
    draw.line([0,200,200,200])

    draw.text((15, 127),"Wave : %s" % waves[wave_index],(255,255,255),font)
    draw.text((15, 207),"Level : %s dBV" % level,(255,255,255),font)
    if wave_index < 5:
        draw.text((15, 167),"Frequency : %shz" % wave_freq,(255,255,255),font)
    elif wave_index > 6:
        draw.text((15, 167),"Frequency : %shz" % oct_freq,(255,255,255),font)

    # B button Arrow
    draw.line([6,140,6,220],width=1)
    draw.line([4,218,6,220,8,218],width=1)

    # A button, Control Resolution
    draw.rectangle([0,0,60,16],fill=colour_palette[3])
    draw.rectangle([0,0,8,75],fill=colour_palette[3])
    draw.line([0,75,8,75,8,16,60,16,60,0],width=1)
    match resolution:
        case 0:
            draw.text((15,0), "Coarse", font=small_font)
        case 1:
            draw.text((15,0), "Fine", font=small_font)
        case 2:
            draw.text((15,0), "X-Fine", font=small_font)



    match active_param:
        case 0:
            draw.rectangle([200,0,240,240],fill=colour_palette[0])
            draw.line([200,160,200,240])
        case 1:
            draw.rectangle([200,0,240,240],fill=colour_palette[1])
            draw.line([200,0,200,160])
            draw.line([200,200,200,240])
        case 2:
            draw.rectangle([200,0,240,240],fill=colour_palette[2])
            draw.line([200,0,200,200])
    
    # draw +
    draw.line([215,70,225,70],width=3)
    draw.line([220,65,220,75],width=3)
    # draw -
    draw.line([215,190,225,190],width=3)



    match wave_index:
        case 0: # Sine
            image.paste(sine_img,[15,25],mask=sine_img)
        case 1: # Triangle
            draw.line([15,70,57.5,25,100,70,142.5,115,185,70],width=2)
        case 2: # Square
            draw.line([15,70,15,25,100,25,100,115,185,115,185,70],width=2)
        case 3: # Saw
            draw.line([15,115,185,25,185,115,185,25],width=2)
        case 4: # Inv Saw
            draw.line([15,115,15,25,185,115,185,25],width=2)
        case 5: # White
            image.paste(white_img,[15,25])
        case 6: # Pink
            image.paste(pink_img,[15,25])
        case 7: # Octaves
            draw_octave_lines(1)
        case 8: # 1/2 Oct
            draw_octave_lines(2)
        case 9: # 1/3 Oct
            draw_octave_lines(3)
        case 10: # Whole Tone
            draw_octave_lines(6)
        case 11: # Semitone
            draw_octave_lines(12)
        

In [ ]:
def nth_octave(n=3,root_freq=440,min_freq=20,max_freq=20000):
    freq_array=[]
    freq=root_freq

    while freq > min_freq:
        freq *=(2**(1/(-n)))

    while freq < max_freq:
        freq *=(2**(1/(n)))
        freq_array.append(freq)
    
    return freq_array

def db_lin(db):
    return 10**(db/20)

def play():
    sc.server.free_all()
    match wave_index:
        case 0: # Sine
            sc.lang.cmds('{SinOsc.ar(%s, 0, %s) }.play' % (wave_freq,db_lin(level)))
        case 1: # Triangle
            sc.lang.cmds('{LFTri.ar(%s,0.0, %s) }.play' % (wave_freq,db_lin(level)))
        case 2: # Square
            sc.lang.cmds('{Pulse.ar(%s, 0.5, %s)}.play' % (wave_freq,db_lin(level)))
        case 3: # Saw
            sc.lang.cmds('{Saw.ar(%s, %s) }.play'% (wave_freq,db_lin(level)))
        case 4: # Inv Saw
            sc.lang.cmds('{Saw.ar(%s, -%s) }.play'% (wave_freq,db_lin(level)))
        case 5: # White
            sc.lang.cmds('{ WhiteNoise.ar(%s)}.play' % db_lin(level))
        case 6: # Pink
            sc.lang.cmds('{ PinkNoise.ar(%s)}.play' % db_lin(level))
        case 7: # Octaves
            sc.lang.cmds('{ Klang.ar(`[%s,%s])}.play' % (nth_octave(1,oct_freq),db_lin(level)))
        case 8: # 1/2 Oct
            sc.lang.cmds('{ Klang.ar(`[%s,%s])}.play' % (nth_octave(2,oct_freq),db_lin(level)/2))
        case 9: # 1/3 Oct
            sc.lang.cmds('{ Klang.ar(`[%s,%s])}.play' % (nth_octave(3,oct_freq),db_lin(level)/3)) 
        case 10: # Whole Tone
            sc.lang.cmds('{ Klang.ar(`[%s,%s])}.play' % (nth_octave(12,oct_freq),db_lin(level)/6))
        case 11: # Semitone
            sc.lang.cmds('{ Klang.ar(`[%s,%s])}.play' % (nth_octave(6,oct_freq),db_lin(level)/12))

In [ ]:
def button_press(button):
    global resolution
    global active_param
    match button:
        case "A":
            resolution +=1
            resolution %=3
            render_display()
            print("inc resolution")
        case "B":
            active_param +=1
            active_param %=3
            render_display()
            print("inc param " +str(active_param))
        case "X":
            match active_param:
                case 0:
                    cycle_wave()
                    render_display()
                    print("inc wave")
                case 1:
                    cycle_freq()
                    render_display()
                    print("inc freq")
                case 2:
                    cycle_level()
                    render_display()
                    print("inc level")
        case "Y":
            match active_param:
                case 0:
                    cycle_wave(False)
                    render_display()
                    print("dec wave")
                case 1:
                    cycle_freq(False)
                    render_display()
                    print("dec freq")
                case 2:
                    cycle_level(False)
                    render_display()
                    print("dec level")
            

In [ ]:
sc.server.free_all()

In [ ]:
button_press("X")
display(image)

In [ ]:
button_press("Y")
display(image)

In [ ]:
button_press("A")
display(image)

In [ ]:
button_press("B")
display(image)